In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('new_222_data_train_norm_decorr.csv')
df_test = pd.read_csv('new_222_data_test_norm_decorr.csv')
df.sample(3)

,prev_year_access_to_exercise_opportunities,prev_year_adult_obesity,prev_year_air_pollution_particulate_matter,prev_year_alcohol_impaired_driving_deaths,prev_year_dentists,prev_year_diabetes_monitoring,prev_year_drinking_water_violations,prev_year_driving_alone_to_work,prev_year_flu_vaccinations,prev_year_injury_deaths,prev_year_long_commute_driving_alone,prev_year_mental_health_providers,prev_year_preventable_hospital_stays,prev_year_primary_care_physicians,prev_year_severe_housing_problems,prev_year_social_associations,prev_year_uninsured,prev_year_violent_crime,coastline_flag,Top_Marginal_Personal_Income_Tax_Rate,"Property_Tax_Burden_(per $1,000 of personal income)","Sales_Tax_Burden(per $1,000 of personal income)","Remaining_Tax_Burden(per $1,000 of personal income)",Estate_Inheritance_Tax_Levied_Flag,Recently_Legislated_Tax_Changes,Debt_Service_as_a_Share_of_Tax_Revenue,"Public_Employees_Per_10,000_of_Population_(full-time equivalent)","State_Liability_System_Survey_(tort litigation treatment, judicial impartiality, etc.)",State_Minimum_Wage_(federal floor is $7.25),Average Workers_Compensation_Costs_(per $100 of payroll),Tax_Expenditure_Limits,GPT_PCA_dim0,GPT_PCA_dim1,GPT_PCA_dim2,GPT_PCA_dim3,GPT_PCA_dim4,GPT_PCA_dim5,GPT_PCA_dim6,GPT_PCA_dim7,GPT_PCA_dim8,GPT_PCA_dim9,GPT_OH_Government_Support,GPT_OH_Large_Customer_Base,GPT_OH_Low_Cost_of_Living,GPT_OH_Natural_Resources,GPT_OH_Supportive_Community,GPT_OH_Limited_Financing,GPT_OH_Limited_Tech_n_Infra,GPT_OH_Low_Pop_Bad_Weather,GPT_OH_Tough_Labor_Market,GPT_OH_Unfavorable_Location,GPT_OH_Various_Downsides,pop_10yr_pct_chg,Prev_Yr_Agri_Pct_GDP,Prev_Yr_Mine_Pct_GDP,Prev_Yr_Retl_Pct_GDP,Prev_Yr_Manu_Pct_GDP,Prev_Yr_Info_Pct_GDP,Prev_Yr_FinS_Pct_GDP,Prev_Yr_EduS_Pct_GDP,Prev_Yr_EntS_Pct_GDP,Prev_Yr_Gvmt_Pct_GDP,Prev_Yr_GoTr_Pct_GDP,two_yrs_prior_pct_it_workers,health_social_pca_0,health_social_pca_1,health_social_pca_2,health_social_pca_3,health_social_pca_4,health_social_pca_5,population_2020,prev_yr_rent_50_2,gpt_ae_pca_0,gpt_ae_pca_1,gpt_ae_pca_2,microbusiness_density
217,0.546476,-1.124314,0.459426,0.233907,-0.120803,0.531494,-0.816345,-2.606003,1.286217,-1.498908,-1.200022,-0.462642,-1.064716,-0.463963,2.285652,-0.461674,-1.083255,-0.362608,3.433091,-1.570762,-0.203350,1.386686,1.829377,1.694905,0.129629,0.877208,-0.326400,0.496113,2.463432,0.474472,1.138042,-0.473382,-0.633677,-0.531071,-0.263820,-0.241616,1.729359,-1.317393,1.493651,0.014433,1.948715,0.652597,-1.202197,0.219221,2.225510,-0.585985,0.674417,1.124355,1.505880,-0.904973,-0.64453,-0.650113,0.934402,0.376812,-0.319608,-0.812432,0.807086,-0.456326,-0.763953,-0.467584,-0.114858,2.060370,-0.533948,-0.513322,-3.174635,2.615523,-0.040484,0.026069,2.350044,0.341553,-0.169924,1.362110,2.935893,-0.293069,0.514044,3.518892
2164,1.503988,-0.263961,0.102052,1.505355,-0.570953,-0.129906,-0.816345,0.029714,0.478758,-0.336960,-1.840961,-0.645224,-1.137301,-0.640826,-0.354032,-0.787806,-0.877973,-0.202556,-0.291167,-1.570762,-0.203350,1.386686,1.829377,1.694905,0.129629,0.877208,-0.326400,0.496113,2.463432,0.474472,1.138042,-0.102281,-1.149380,-0.828245,0.075613,0.896492,1.252167,-0.569878,0.708552,0.199701,1.141321,0.652597,-1.202197,0.219221,-0.449156,1.705847,0.674417,-0.889044,1.505880,-0.904973,1.55090,-0.650113,0.385466,-0.363462,-0.325416,1.927790,-0.600532,-0.300635,0.584128,-1.357832,0.430417,-0.154496,-0.533948,-0.111166,-1.504802,-0.370567,0.018830,-0.261346,0.966634,0.065619,-0.244370,-0.226970,2.500051,-2.124414,-0.617688,2.690792
1813,-2.195489,-0.091890,0.935925,0.033152,1.142239,0.035444,-0.816345,0.307158,-1.237092,-1.461426,0.883029,1.522354,1.162127,2.317217,-0.114061,-1.457236,1.380133,-0.821424,-0.291167,0.190952,-0.288501,-0.441997,-1.329423,-0.590063,-0.431182,-0.202188,-0.661558,-0.550229,-0.807349,0.776382,-0.982727,1.568773,-0.102117,-0.293881,-0.427854,0.023526,0.253705,-0.108804,0.304157,1.285309,0.125951,0.652597,0.831479,0.219221,-0.449156,-0.585985,-1.482170,-0.889044,-0.663799,-0.904973,1.

In [3]:
X = df.loc[:, df.columns != 'microbusiness_density']
y = df['microbusiness_density']
y = np.log(y)

X_test = df_test.loc[:, df_test.columns != 'microbusiness_density']
y_test = df_test['microbusiness_density']
y_test = np.log(y_test)

In [4]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

In [5]:
best_K = 15
best_N_features = 20

sfs = SequentialFeatureSelector(KNeighborsRegressor(n_neighbors=best_K),
                                n_features_to_select=best_N_features,
                                direction='forward',
                                n_jobs=-1)
sfs.fit(X, y)
selected = sfs.get_support(indices=True)
X_selected = X.iloc[:, selected]
print(X_selected.columns)

Index(['prev_year_adult_obesity', 'prev_year_dentists',
       'prev_year_primary_care_physicians',
       'prev_year_severe_housing_problems',
       'Property_Tax_Burden_(per $1,000 of personal income)',
       'State_Minimum_Wage_(federal floor is $7.25)', 'pop_10yr_pct_chg',
       'Prev_Yr_Agri_Pct_GDP', 'Prev_Yr_Retl_Pct_GDP', 'Prev_Yr_Info_Pct_GDP',
       'Prev_Yr_FinS_Pct_GDP', 'Prev_Yr_EduS_Pct_GDP', 'Prev_Yr_EntS_Pct_GDP',
       'two_yrs_prior_pct_it_workers', 'health_social_pca_0',
       'health_social_pca_1', 'health_social_pca_2', 'health_social_pca_3',
       'population_2020', 'prev_yr_rent_50_2'],
      dtype='object')


In [6]:
knn = KNeighborsRegressor(n_neighbors=best_K)
rmse = np.sqrt(-cross_val_score(knn, X_selected, y, cv=5, scoring='neg_mean_squared_error'))
print('K =', best_K, 'RMSE =', round(rmse.mean(),3), '±', round(rmse.std(),3))

K = 15 RMSE = 0.449 ± 0.039


In [7]:
# test set
knn.fit(X_selected, y)
rmse_test = np.sqrt(np.mean((y_test - knn.predict(X_test.iloc[:, selected]))**2))
print('K =', best_K, 'RMSE =', round(rmse_test,3))

K = 15 RMSE = 0.417
